# Demo: scipy.stats & statsmodels
This notebook demonstrates common hypothesis tests (t-test, chi-square, one-way ANOVA) using `scipy.stats`, and regression + diagnostics (OLS, VIF, Durbin-Watson, Breusch–Pagan) using `statsmodels`.

You can run this notebook locally or review the code snippets and outputs.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

np.random.seed(0)


In [ ]:
# Generate synthetic data for examples
group_a = np.random.normal(loc=5.0, scale=1.0, size=50)
group_b = np.random.normal(loc=5.5, scale=1.2, size=55)

contingency = np.array([[30, 10], [20, 40]])

g1 = np.random.normal(5.0, 1.0, size=30)
g2 = np.random.normal(5.4, 1.1, size=28)
g3 = np.random.normal(4.8, 0.9, size=32)

n = 200
x1 = np.random.normal(10, 2, size=n)
x2 = np.random.normal(5, 1.5, size=n)
x3 = x1 * 0.8 + np.random.normal(0, 0.5, size=n)
y = 2.0 + 0.5 * x1 - 0.3 * x2 + 0.1 * x3 + np.random.normal(0, 1.0, size=n)

df = pd.DataFrame({'y': y, 'x1': x1, 'x2': x2, 'x3': x3})
df.head()


In [ ]:
# Independent t-test (Welch's t-test)
tstat, pval = stats.ttest_ind(group_a, group_b, equal_var=False)
print('t-statistic:', tstat)
print('p-value:', pval)


In [ ]:
# Chi-square test of independence
chi2, p, dof, expected = stats.chi2_contingency(contingency)
print('chi2:', chi2)
print('p-value:', p)
print('degrees of freedom:', dof)
print('expected frequencies:\n', expected)


In [ ]:
# One-way ANOVA
fstat, p_anova = stats.f_oneway(g1, g2, g3)
print('F-statistic:', fstat)
print('p-value:', p_anova)


In [ ]:
# OLS regression with statsmodels
X = df[['x1','x2','x3']]
X = sm.add_constant(X)
model = sm.OLS(df['y'], X).fit()
print(model.summary())


In [ ]:
# VIF to check multicollinearity
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data


In [ ]:
# Durbin-Watson for autocorrelation of residuals
dw = durbin_watson(model.resid)
print('Durbin-Watson:', dw)

# Breusch-Pagan test for heteroscedasticity
bp_test = het_breuschpagan(model.resid, model.model.exog)
bp_labels = ['Lagrange multiplier stat', 'LM p-value', 'f-value', 'f p-value']
print(dict(zip(bp_labels, bp_test)))
